# Ollama Legal PDF RAG Notebook

## Import Libraries


In [1]:
# Imports
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_ollama import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_ollama.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

# Jupyter-specific imports
from IPython.display import display, Markdown

# Set environment variable for protobuf
import os
os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"] = "python"

## Load PDF
change the path to document for different case files

In [2]:
# Load PDF

local_path = "sample_data\Ram_Sarup_Gupta_Dead_By_Lrs_vs_Bishun_Narain_Inter_College_Ors_on_8_April_1987.PDF"
if local_path:
    loader = UnstructuredPDFLoader(file_path=local_path)
    data = loader.load()
    print(f"PDF loaded successfully: {local_path}")
else:
    print("Upload a PDF file")

PDF loaded successfully: sample_data\Ram_Sarup_Gupta_Dead_By_Lrs_vs_Bishun_Narain_Inter_College_Ors_on_8_April_1987.PDF


In [3]:
#print(data)

## Split text into chunks

In [4]:
# Split text into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=3000, chunk_overlap=200)
chunks = text_splitter.split_documents(data)
print(f"Text split into {len(chunks)} chunks")

Text split into 31 chunks


In [5]:
#print(chunks)  # Display first 500 characters of the first chunk

## Create vector database

In [6]:
# Create vector database
vector_db = Chroma.from_documents(
    documents=chunks,
    embedding=OllamaEmbeddings(model="nomic-embed-text"),
    collection_name="local-rag"
)
print("Vector database created successfully")

Vector database created successfully


## Set up LLM and Retrieval

In [7]:
# Set up LLM and retrieval
local_model = "gemma3:4b"  # or whichever model you prefer
llm = ChatOllama(model=local_model)

In [8]:
# Query prompt template
QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant. Your task is to generate 2
    different versions of the given user question to retrieve relevant documents from
    a vector database. By generating multiple perspectives on the user question, your
    goal is to help the user overcome some of the limitations of the distance-based
    similarity search. Provide these alternative questions separated by newlines.
    Original question: {question}""",
)

# Set up retriever
retriever = MultiQueryRetriever.from_llm(
    vector_db.as_retriever(), 
    llm,
    prompt=QUERY_PROMPT
)

## Create chain

In [9]:
# RAG prompt template
template = """
You are a legal assistant. Use ONLY the information provided in the context below to answer the question as accurately and concisely as possible. Do not make up any information or provide legal advice.

Context:
{context}

Question:
{question}

Answer (based strictly on the context):
"""

prompt = ChatPromptTemplate.from_template(template)

In [10]:
# Create chain
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

## Chat with PDF

In [11]:
def chat_with_pdf(question):
    """
    Chat with the PDF using the RAG chain.
    """
    return display(Markdown(chain.invoke(question)))

In [12]:
chat_with_pdf("What is the main idea of Irrevocable License for School: Ram Sarup Gupta vs Bishun Narain?")

The main idea of the Ram Sarup Gupta vs. Bishun Narain case is that a licensee (the school) can obtain an irrevocable license under Section 60(b) of the Indian Easement Act, 1882, even if they have built permanent structures and incurred expenses on the land, *provided* they clearly plead and prove that these constructions and expenses were made “acting upon the license.” The court emphasized the importance of specific pleading and evidence to support the claim of irrevocability, rather than simply relying on the execution of permanent works. The school failed to adequately plead this crucial element, leading to the court dismissing their claim.

In [13]:
chat_with_pdf("Summarize the arguments regarding the irrevocability of the license.")

The core argument revolves around whether the school’s construction on the property constituted a permanent, irrevocable license under Section 60(b) of the Indian Easements Act. Here’s a breakdown of the key arguments:

**School’s Argument (for Irrevocability):**

*   **Permanent Construction:** The school erected works of a permanent nature (buildings) on the property.
*   **“Acting Upon the License”:** They did so while utilizing the license for the purpose of running the school.
*   **Expenses Incurred:** They spent money on these constructions as part of operating the school.

**Opposing Argument (against Irrevocability):**

*   **Lack of Specific Pleadings:** The opposing side argued the school failed to present sufficient pleadings to demonstrate the license was truly irrevocable. They contended that the courts shouldn't have created a new case for the defendants.
*   **Absence of Issue Framed:** No specific issue was framed concerning the irrevocability of the license.

**Court’s Decision (Based on the provided text):**

The court ultimately ruled in favor of the school, stating that all three conditions – permanent construction, "acting upon the license," and incurred expenses – were met, making the license irrevocable. The court emphasized that allowing someone to build on land while granting a license doesn’t automatically create a right to revoke it later. 

**In essence, the arguments hinged on whether the school’s actions fully satisfied the requirements for an irrevocable license under Section 60(b) of the Easements Act.**

In [14]:
chat_with_pdf("Can you explain the case study highlighted in the document?")

The case study revolves around a dispute concerning an irrevocable license granted to Bishun Narain Inter College (the respondents) to build on land owned by Ram Surup Gupta (the appellant). The core issue was whether the license was, in fact, irrevocable.

Here's a breakdown of the key points:

* **The Dispute:** Gupta argued that the respondents’ construction and incurring expenses constituted a permanent structure and therefore, the license was irrevocable under Section 60(b) of the Indian Easement Act, 1882.
* **Respondents’ Argument:** The respondents contended that their construction and expenses made the license irrevocable.
* **Court’s Decision:** The court, led by Justice T.S. Misra, sided with Gupta.  The court found that the respondents failed to adequately plead their case. They hadn’t produced evidence demonstrating that the construction and expenses were “acting upon the license” – a crucial element required to make the license irrevocable. The court emphasized that the pleadings need not mirror the exact wording of the statute and focused on the substance of the case.  The judges agreed that the respondents’ pleadings were insufficient to support their claim.

In essence, the case highlights the importance of proper pleading in legal proceedings, particularly when claiming rights based on statutory provisions. Simply executing work or incurring expenses isn't enough; the claim must be clearly and adequately presented in the pleadings.

In [15]:
chat_with_pdf("tell me the key points in the document")

Here’s a breakdown of the key points from the document:

*   **Lack of Express Pleadings Doesn't Necessarily Bar a Claim:** The core argument is that even though the plaintiff (Bishun Narain Inter College) didn’t explicitly plead that the license was perpetual and irrevocable, the court can still consider it if it’s supported by evidence.

*   **Substance Over Form:** The court emphasizes that the focus should be on the *substance* of the pleadings and the issues raised, rather than strictly adhering to the form of the pleadings.

*   **Knowledge of the Issue:** A crucial factor is whether the parties *knew* the issue was being contested. If both parties were aware of a particular matter and evidence was presented about it, a party can rely on it even if it wasn't explicitly pleaded.

*   **Evidence is Key:**  If evidence is presented regarding a matter, a party can invoke it, regardless of whether it was specifically pleaded. The plaintiff did this by producing the donor in evidence to support his claim of an irrevocable license.

*   **Liberal Construction of Pleadings:** The court suggests a liberal approach to interpreting pleadings, avoiding pedantic focus on minor technicalities.

*   **Bhagwati Prasad Case Analogy:** The court refers to the *Bhagwati Prasad v. Shri Chandramaul* case, which provides a similar principle – a plea isn’t denied simply because it wasn't expressly made if the issue was known and evidence was presented.

In essence, the document establishes a principle that courts will look beyond the strict wording of pleadings when evidence demonstrates a party's claim.

## Chatbot UI

In [16]:
import gradio as gr
import nest_asyncio
nest_asyncio.apply()

def gradio_interface(question, history):
    try:
        response = chain.invoke(question)
    except Exception:
        response = "⚠️ Sorry, something went wrong while processing your request."
    return response

# Gradio UI
demo = gr.ChatInterface(
    fn=gradio_interface,
    title="Legal Document Chatbot",
    description="Ask questions about the Document",
    examples=[
        "tell me the key points in the document.",
        "Summarize the court's reasoning on this case."
    ]
)

# ⚠️ Safely launch Gradio without showing tracebacks
try:
    demo.launch(share=True, show_error=False)  # `show_error=False` hides internal errors
except Exception:
    pass  


* Running on local URL:  http://127.0.0.1:7860


## Clean up (optional)

In [17]:
# Optional: Clean up when done 
#vector_db.delete_collection()
#print("Vector database deleted successfully")